In [1]:
%cd ..

/home/nikita/edu/competitions/admet


In [2]:
import pandas as pd
import datamol as dm
import numpy as np
import warnings

from molfeat.calc import RDKitDescriptors2D, FPCalculator, MordredDescriptors
from molfeat.trans import MoleculeTransformer

from molfeat.trans.concat import FeatConcat
from molfeat.trans.fp import FPVecTransformer

from rdkit import Chem, RDLogger
from rdkit.Chem import Descriptors
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator


from mordred import Calculator, descriptors
import rdkit


from tqdm import tqdm

warnings.filterwarnings("ignore")

In [3]:
train_data = dm.read_csv("data/train_admet_pumped.csv", index_col=0, smiles_column="Drug")

[07:53:59] WARNING: not removing hydrogen atom without neighbors
[07:53:59] WARNING: not removing hydrogen atom without neighbors
[07:53:59] WARNING: not removing hydrogen atom without neighbors
[07:53:59] WARNING: not removing hydrogen atom without neighbors
[07:53:59] WARNING: not removing hydrogen atom without neighbors
[07:53:59] WARNING: not removing hydrogen atom without neighbors
[07:53:59] WARNING: not removing hydrogen atom without neighbors
[07:53:59] WARNING: not removing hydrogen atom without neighbors
[07:53:59] WARNING: not removing hydrogen atom without neighbors
[07:53:59] WARNING: not removing hydrogen atom without neighbors
[07:53:59] WARNING: not removing hydrogen atom without neighbors
[07:53:59] WARNING: not removing hydrogen atom without neighbors
[07:53:59] WARNING: not removing hydrogen atom without neighbors
[07:53:59] WARNING: not removing hydrogen atom without neighbors
[07:53:59] WARNING: not removing hydrogen atom without neighbors
[07:53:59] WARNING: not r

In [4]:
d2 = FPVecTransformer("desc2D", n_jobs=8, dtype=np.float32, replace_nan=True)
maccs = FPVecTransformer("maccs", dtype=np.float32)
ecfp4 = FPVecTransformer("ecfp:4", dtype=np.float32)

# Wrap the calculator in a transformer instance
featurizer = FeatConcat([maccs, ecfp4, d2], dtype=np.float32)

with dm.without_rdkit_log():
    feats = featurizer(train_data["mol"])

[07:54:16] DEPRECATION WARNING: please use MorganGenerator
[07:54:16] DEPRECATION WARNING: please use MorganGenerator
[07:54:16] DEPRECATION WARNING: please use MorganGenerator
[07:54:16] DEPRECATION WARNING: please use MorganGenerator
[07:54:16] DEPRECATION WARNING: please use MorganGenerator
[07:54:16] DEPRECATION WARNING: please use MorganGenerator
[07:54:16] DEPRECATION WARNING: please use MorganGenerator
[07:54:16] DEPRECATION WARNING: please use MorganGenerator
[07:54:16] DEPRECATION WARNING: please use MorganGenerator
[07:54:16] DEPRECATION WARNING: please use MorganGenerator
[07:54:16] DEPRECATION WARNING: please use MorganGenerator
[07:54:16] DEPRECATION WARNING: please use MorganGenerator
[07:54:17] DEPRECATION WARNING: please use MorganGenerator
[07:54:17] DEPRECATION WARNING: please use MorganGenerator
[07:54:17] DEPRECATION WARNING: please use MorganGenerator
[07:54:17] DEPRECATION WARNING: please use MorganGenerator
[07:54:17] DEPRECATION WARNING: please use MorganGenerat

In [5]:
warnings.filterwarnings("ignore")
RDLogger.DisableLog("rdApp.*")


def get_decsriptors_df(smiles_list):
    descriptors_list = []

    for smiles in tqdm(smiles_list):
        descriptors_list.append(
            Descriptors.CalcMolDescriptors(Chem.MolFromSmiles(smiles), 0)
        )
    return pd.DataFrame(descriptors_list).fillna(0)

rdkit_feats = get_decsriptors_df(train_data["Drug"])

100%|██████████| 10299/10299 [02:48<00:00, 61.03it/s] 


In [6]:
calc = Calculator(descriptors, ignore_3D=False)
mols = [rdkit.Chem.MolFromSmiles(smiles) for smiles in train_data["Drug"]]
desc_vector = calc.pandas(mols).astype(float).fillna(0)

100%|██████████| 10299/10299 [16:31<00:00, 10.39it/s] 


In [7]:
def fp_str_to_array(str_): 
    return np.array([int(bit) for bit in list(str_)]) 
 
def get_decsriptors_w_fp_df(smiles_list): 
    descriptors_list = [] 
    fpgen = GetMorganGenerator(radius=4, 
                               fpSize=64) 
 
    for smiles in tqdm(smiles_list): 
        fp = fp_str_to_array(fpgen.GetFingerprint(Chem.MolFromSmiles(smiles)).ToBitString()) 
        descriptors_list.append(list(Descriptors.CalcMolDescriptors( 
            Chem.MolFromSmiles(smiles), 0).values()) + fp.tolist()) 
    return pd.DataFrame(descriptors_list).fillna(0)

In [8]:
fingerprints = get_decsriptors_w_fp_df(train_data["Drug"].tolist())

  0%|          | 0/10299 [00:00<?, ?it/s]

100%|██████████| 10299/10299 [01:11<00:00, 144.24it/s]


In [9]:
feats = pd.DataFrame(feats, columns=[f"{i}" for i in range(feats.shape[1])])

In [10]:
total_data = pd.concat([train_data, feats, rdkit_feats, desc_vector, fingerprints], axis=1)
total_data.drop(["mol"], axis=1, inplace=True)

In [11]:
total_data

,Drug_ID,Drug,Y,property,0,1,2,3,4,5,...,264,265,266,267,268,269,270,271,272,273
0,0,O=[N+]([O-])c1c2c(c3ccc4cccc5ccc1c3c45)CCCC2,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,1,0,1,0,1,0,1
1,1,O=c1c2ccccc2c(=O)c2c1ccc1c2[nH]c2c3c(=O)c4cccc...,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,1,0,1,0,1,0,0,1
2,2,[N-]=[N+]=CC(=O)NCC(=O)NN,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,1,0,0
3,3,[N-]=[N+]=C1C=NC(=O)NC1=O,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,1,0,0,0,0,0
4,4,CCCCN(CC(O)C1=CC(=[N+]=[N-])C(=O)C=C1)N=O,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,1,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10294,25526,CCC(C)(CNC(=O)CCCO)c1cccc(OCC)c1,1,3,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,0,0,0,0,1,1,0
10295,25530,CCCC(CC)(O)C1CC23C=CC1(OCC)C1Oc4c(O)ccc5c4C12C...,1,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,1,0,1,1,1,1,1
10296,25534,COC(OCC)C(=O)[C@@]12OC(CC)(CC)O[C@@H]1CC1C3C[C...,1,3,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,0,1,1,1,1,1,1
10297,25538,CCC(CC)(O)C1CC23C=CC1(OCC)C1Oc4c(OCC)ccc5c4C12...,1,3,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,1,1,1,1,0


In [20]:
total_data.to_csv("data/train_admet_pumped_with_all_new_features.csv", index=False)